In [1]:
from lm_eval import simple_evaluate
from lm_eval.utils import make_table

/home/louis/miniconda3/envs/mamba2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from transformers import AutoTokenizer

# 1. Define model and tokenizer IDs
model_id = "state-spaces/mamba2-1.3b"
tokenizer_id = "EleutherAI/gpt-neox-20b" # Standard tokenizer for Mamba models

# 2. Load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

# 3. Load the Model using the official mamba-ssm library
print(f"Loading {model_id}...")
model = MambaLMHeadModel.from_pretrained(
    model_id, 
    device="cuda", 
    dtype=torch.bfloat16
)

Loading state-spaces/mamba2-1.3b...


In [3]:
# Quick generation test
text = "The capital of France is"
input_ids = tokenizer(text, return_tensors="pt").input_ids.to("cuda")

print(f"Input: {text}")
out = model.generate(
    input_ids=input_ids, 
    max_length=30, 
    temperature=0.7, 
    top_p=0.9
)
print(f"Output: {tokenizer.decode(out[0])}")

Input: The capital of France is
Output: The capital of France is a city of contrasts. It is a city of the past, a city of the present, and a city of the future


In [4]:
# First, create the HFLM wrapper
from lm_eval.models.huggingface import HFLM

class MambaHFLM(HFLM):
    """Custom wrapper for mamba-ssm models"""
    
    def _model_generate(self, context, max_length, stop, **generation_kwargs):
        # Remove kwargs that mamba-ssm doesn't support
        generation_kwargs.pop('stopping_criteria', None)
        generation_kwargs.pop('pad_token_id', None)
        generation_kwargs.pop('use_cache', None)
        
        with torch.no_grad():
            return self.model.generate(
                input_ids=context,
                max_length=max_length,
                temperature=generation_kwargs.get('temperature', 1.0),
                top_p=generation_kwargs.get('top_p', 1.0),
                eos_token_id=self.tokenizer.eos_token_id,
            )

# Patch mamba-ssm model for HFLM compatibility
model.device = next(model.parameters()).device

class MambaConfig:
    vocab_size = 50277
    hidden_size = 2048
    num_hidden_layers = 48
    tie_embeddings = True

model.config = MambaConfig()
model.tie_weights = lambda: None

# Create the wrapper
print("🔌 Creating MambaHFLM wrapper...")
lm_obj = MambaHFLM(
    pretrained=model,
    tokenizer=tokenizer,
    batch_size=1,
    max_length=131072,
    trust_remote_code=True
)
print(f"✅ Wrapper created. Max length: {lm_obj.max_length}")

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration


🔌 Creating MambaHFLM wrapper...
✅ Wrapper created. Max length: 131072


In [32]:
# Try niah_single_1 (pass-key retrieval) which might match the paper
print("🧪 TESTING niah_single_1 (pass-key retrieval)")
print("=" * 60)

results = simple_evaluate(
    model=lm_obj,
    tasks=["niah_single_1"],
    device="cuda",
    num_fewshot=0,
    limit=20,
    metadata={"max_seq_lengths": [4096], "tokenizer": tokenizer_id}
)

print(make_table(results))

🧪 TESTING niah_single_1 (pass-key retrieval)


niah_single_1: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: repeat | 4096: 100%|███████████████████████████████████| 500/500 [00:03<00:00, 153.34it/s]
Overwriting default num_fewshot of niah_single_1 from None to 0
Running generate_until requests: 100%|██████████████████████████████████████████████████| 20/20 [01:48<00:00,  5.43s/it]
fatal: not a git repository (or any of the parent directories): .git


|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_1|      1|none  |     0|  4096|↑  |    1|±  |   N/A|



In [ ]:
TASK = "niah_single_2"
LENGTHS = [1024, 2048, 4096, 8192]

results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK],
    device="cuda",
    num_fewshot=0,
    
    metadata={"max_seq_lengths": LENGTHS, "tokenizer": tokenizer_id}
)
print(make_table(results))

niah_single_2: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 1024: 100%|████████████████████████████████████| 500/500 [00:00<00:00, 913.94it/s]
Generating synthetic samples: essay | 2048: 100%|████████████████████████████████████| 500/500 [00:02<00:00, 229.54it/s]
Generating synthetic samples: essay | 4096: 100%|████████████████████████████████████| 500/500 [00:03<00:00, 147.73it/s]
Generating synthetic samples: essay | 8192: 100%|█████████████████████████████████████| 500/500 [00:06<00:00, 78.96it/s]
Overwriting default num_fewshot of niah_single_2 from None to 0
Running generate_until requests: 100%|████████████████████████████████████████████| 2000/2000 [2:50:24<00:00,  5.11s/it]


|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_2|      1|none  |     0|  1024|   |0.990|±  |0.0045|
|             |       |none  |     0|  2048|   |0.768|±  |0.0189|
|             |       |none  |     0|  4096|↑  |0.000|±  |   N/A|
|             |       |none  |     0|  8192|↑  |0.000|±  |   N/A|



In [17]:
TASK = "niah_single_1"
LENGTHS = [1024, 2048, 4096, 8192]

results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK],
    device="cuda",
    num_fewshot=0,
    metadata={"max_seq_lengths": LENGTHS, "tokenizer": tokenizer_id}
)
print(make_table(results))

niah_single_1: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: repeat | 1024: 100%|███████████████████████████████████| 500/500 [00:00<00:00, 775.15it/s]
Generating synthetic samples: repeat | 2048: 100%|███████████████████████████████████| 500/500 [00:01<00:00, 338.11it/s]
Generating synthetic samples: repeat | 4096: 100%|███████████████████████████████████| 500/500 [00:03<00:00, 164.75it/s]
Generating synthetic samples: repeat | 8192: 100%|████████████████████████████████████| 500/500 [00:05<00:00, 86.59it/s]
Overwriting default num_fewshot of niah_single_1 from None to 0
Running generate_until requests: 100%|████████████████████████████████████████████| 2000/2000 [2:48:42<00:00,  5.06s/it]


|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_1|      1|none  |     0|  1024|   |    1|±  |     0|
|             |       |none  |     0|  2048|   |    1|±  |     0|
|             |       |none  |     0|  4096|↑  |    1|±  |   N/A|
|             |       |none  |     0|  8192|↑  |    1|±  |   N/A|



In [18]:
TASK = "niah_single_3"
LENGTHS = [1024, 2048, 4096, 8192]

results = simple_evaluate(
    model=lm_obj,
    tasks=[TASK],
    device="cuda",
    num_fewshot=0,
    metadata={"max_seq_lengths": LENGTHS, "tokenizer": tokenizer_id}
)
print(make_table(results))

niah_single_3: Custom kwargs can be passed to `--metadata` in console (as json string) or to the TaskManager.
For example --metadata='{"max_seq_lengths":[4096, 8192]}'. For details see task Readme.
Generating synthetic samples: essay | 1024: 100%|████████████████████████████████████| 500/500 [00:00<00:00, 801.77it/s]
Generating synthetic samples: essay | 2048: 100%|████████████████████████████████████| 500/500 [00:01<00:00, 433.00it/s]
Generating synthetic samples: essay | 4096: 100%|████████████████████████████████████| 500/500 [00:03<00:00, 147.80it/s]
Generating synthetic samples: essay | 8192: 100%|█████████████████████████████████████| 500/500 [00:06<00:00, 73.65it/s]
Overwriting default num_fewshot of niah_single_3 from None to 0
Running generate_until requests: 100%|████████████████████████████████████████████| 2000/2000 [2:48:21<00:00,  5.05s/it]


|    Tasks    |Version|Filter|n-shot|Metric|   |Value|   |Stderr|
|-------------|------:|------|-----:|-----:|---|----:|---|------|
|niah_single_3|      1|none  |     0|  1024|   |0.982|±  |0.0060|
|             |       |none  |     0|  2048|   |0.810|±  |0.0176|
|             |       |none  |     0|  4096|↑  |0.002|±  |   N/A|
|             |       |none  |     0|  8192|↑  |0.000|±  |   N/A|

